In [43]:
import os
import pandas as pd
import mysql.connector as mysql
from mysql.connector import Error


def DBConnect(dbName=None):
    """
    Parameters
    ----------
    dbName :
        Default value = None)
    Returns
    -------
    """
    # conn = mysql.connect(host='localhost', user='root', password=os.getenv('mysqlPass'),
    #                      database=dbName, buffered=True)

    conn = mysql.connect(host='localhost', user='aman', password='1234',
                         database=dbName, buffered=True)
    cur = conn.cursor()
    return conn, cur


def emojiDB(dbName: str) -> None:
    conn, cur = DBConnect(dbName)
    dbQuery = f"ALTER DATABASE {dbName} CHARACTER SET = utf8mb4 COLLATE = utf8mb4_unicode_ci;"
    cur.execute(dbQuery)
    conn.commit()


def createDB(dbName: str) -> None:
    """
    Parameters
    ----------
    dbName :
        str:
    dbName :
        str:
    dbName:str :
    Returns
    -------
    """
    conn, cur = DBConnect()
    cur.execute(f"CREATE DATABASE IF NOT EXISTS {dbName};")
    conn.commit()
    cur.close()


def createTables(dbName: str) -> None:
    """
    Parameters
    ----------
    dbName :
        str:
    dbName :
        str:
    dbName:str :
    Returns
    -------
    """
    conn, cur = DBConnect(dbName)
    sqlFile = 'schema.sql'
    fd = open(sqlFile, 'r')
    readSqlFile = fd.read()
    fd.close()

    sqlCommands = readSqlFile.split(';')
    for command in sqlCommands:
        try:
            res = cur.execute(command)
        except Exception as ex:
            print("Command skipped: ", command)
            print(ex)
    conn.commit()
    cur.close()

    return


def preprocess_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Parameters
    ----------
    df :
        pd.DataFrame:
    df :
        pd.DataFrame:
    df:pd.DataFrame :
    Returns
    -------
    """

    df['possibly_sensitive'] = df['possibly_sensitive'].fillna(0)
    df['place'] = df['place'].fillna(" ")

    df['hashtags'] = df['hashtags'].fillna(" ")
    df['user_mentions'] = df['user_mentions'].fillna(" ")

    df['retweet_count'] = df['retweet_count'].fillna(0)
    # try:
    #     df = df.drop(["user_mentions"],
    #                  axis='columns')
    # except KeyError as e:
    #     print("Error:", e)

    return df


def insert_to_tweet_table(dbName: str, df: pd.DataFrame, table_name: str) -> None:
    """
    Parameters
    ----------
    dbName :
        str:
    df :
        pd.DataFrame:
    table_name :
        str:
    dbName :
        str:
    df :
        pd.DataFrame:
    table_name :
        str:
    dbName:str :
    df:pd.DataFrame :
    table_name:str :
    Returns
    -------
    """
    conn, cur = DBConnect(dbName)

    df = preprocess_df(df)

    for _, row in df.iterrows():
        sqlQuery = f"""INSERT INTO {table_name} (created_at, source, original_text, polarity, subjectivity, lang,
                    favorite_count, retweet_count, original_author, followers_count, friends_count, possibly_sensitive,
                    hashtags, user_mentions, place, clean_text)
             VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"""
        data = (row[0], row[1], row[2], row[3], (row[4]), (row[5]), row[6], row[7], row[8], row[9], row[10], row[11],
                row[12], row[13], row[14], row[15])

        try:
            # Execute the SQL command
            cur.execute(sqlQuery, data)
            # Commit your changes in the database
            conn.commit()
            print("Data Inserted Successfully")
        except Exception as e:
            conn.rollback()
            print("Error: ", e)
    return


def db_execute_fetch(*args, many=False, tablename='', rdf=True, **kwargs) -> pd.DataFrame:
    """
    Parameters
    ----------
    *args :
    many :
         (Default value = False)
    tablename :
         (Default value = '')
    rdf :
         (Default value = True)
    **kwargs :
    Returns
    -------
    """
    connection, cursor1 = DBConnect(**kwargs)
    if many:
        cursor1.executemany(*args)
    else:
        cursor1.execute(*args)

    # get column names
    field_names = [i[0] for i in cursor1.description]

    # get column values
    res = cursor1.fetchall()

    # get row count and show info
    nrow = cursor1.rowcount
    if tablename:
        print(f"{nrow} recrods fetched from {tablename} table")

    cursor1.close()
    connection.close()

    # return result
    if rdf:
        return pd.DataFrame(res, columns=field_names)
    else:
        return res


if __name__ == "__main__":
    createDB(dbName='tweets')
    emojiDB(dbName='tweets')
    createTables(dbName='tweets')

    df = pd.read_csv('../cleanedPre_gl_tweeter_data_sample.csv')
    df.info()

    insert_to_tweet_table(dbName='tweets', df=df,
                          table_name='TweetInformation')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21997 entries, 0 to 21996
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   created_at          4 non-null      object 
 1   source              4 non-null      object 
 2   original_text       4 non-null      object 
 3   polarity            4 non-null      float64
 4   subjectivity        4 non-null      float64
 5   lang                4 non-null      object 
 6   favorite_count      4 non-null      float64
 7   retweet_count       4 non-null      float64
 8   original_author     4 non-null      object 
 9   followers_count     4 non-null      float64
 10  friends_count       4 non-null      float64
 11  possibly_sensitive  0 non-null      float64
 12  hashtags            4 non-null      object 
 13  user_mentions       3 non-null      object 
 14  place               2 non-null      object 
 15  clean_text          4 non-null      object 
dtypes: f

Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field lis

Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field lis

Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field lis

Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field lis

...

Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field lis

Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field lis

Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field lis

Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field lis

Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field list'
Error:  1054 (42S22): Unknown column 'nan' in 'field lis